<a href="https://colab.research.google.com/github/MileneBedouhene/Face-Recognition-Using-CNN/blob/main/Challenge2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
TrainingImagePath= '/content/drive/MyDrive/Me/AI/DeepLearning/Challenges/FaceImages/FinalTrainingImages'
TestingImagePath= '/content/drive/MyDrive/Me/AI/DeepLearning/Challenges/FaceImages/FinalTestingImages'

In [ ]:
# Defining pre-processing transformations on raw images of training data
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# No transformations are done on the testing images
test_datagen = ImageDataGenerator()

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Generating the Testing Data
testing_set = test_datagen.flow_from_directory(
        TestingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

In [ ]:
# Displays a dictionary that associates each class with a numeric index
testing_set.class_indices

#### Creating lookup table for all faces


In [ ]:
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

# Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)


print("Mapping of Face and its ID",ResultMap)
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

#### Building The CNN

In [ ]:
#Create CNN deep learning model
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
#Initializing the Convolutional Neural Network
classifier= Sequential()

#STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend

classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

# STEP--2 MAX Pooling
classifier.add(MaxPool2D(pool_size=(2,2)))

#ADDITIONAL LAYER of CONVOLUTION
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))

# MAX Polling
classifier.add(MaxPool2D(pool_size=(2,2)))

# STEP--3 FLattening
classifier.add(Flatten())

# STEP--4 Fully Connected Neural Network
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(OutputNeurons, activation='softmax'))


In [ ]:
classifier.summary()

In [ ]:
# Compiling the CNN
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

In [ ]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit(
              training_set,
              steps_per_epoch=30,
              epochs=20)

EndTime=time.time()
print("Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')

In [ ]:
import numpy as np
from keras.preprocessing import image

In [ ]:
#Making single predictions


ImagePath='/content/drive/MyDrive/Me/AI/DeepLearning/Challenges/FaceImages/FinalTestingImages/face6/3face6.jpg'
test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)


print('Prediction is: ',ResultMap[np.argmax(result)])